In [22]:
!pip install openpyxl 
!pip install geopandas
!pip install shapely




[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [99]:
import warnings
# Ignorar todas las advertencias (NO RECOMENDADO en producción) por practicidad
warnings.filterwarnings("ignore")

In [86]:
import pandas as pd
import geopandas as gpd
import matplotlib_inline as plt
import numpy as np
import json 
from shapely.geometry import shape, Polygon, Point
from scipy.spatial.distance import euclidean


In [3]:
poblacion_sec_censal = pd.read_csv('/work/GPX programing test/poblacion_sec_censal.csv', sep= ';')
#population of all census sections of the province of Barcelona identified by their unique census section identifier.
poblacion_sec_censal.head(10)


total_poblacion = poblacion_sec_censal['Total'].sum()

print(total_poblacion)

280703.167


In [18]:
atlas_socioeconimo = pd.read_excel('/work/GPX programing test/atlas_socioeconomico.xlsx', header=7)
#Here we have to face the design of the sheet and only extract the headers that start at row 7 

In [19]:
atlas_socioeconimo.head(10)

,,2017,2016,2015,2017.1,2016.1,2015.1,2017.2,2016.2,2015.2,2017.3,2016.3,2015.3,2017.4,2016.4,2015.4
0,01001 Alegría-Dulantzi,13281,13086,12936,34618,34373,33702,,,,,,,,,
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,,,,,,
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,,,,,,
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,,,,,,
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,
5,0100201 Amurrio distrito 01,,,,,,,,,,,,,,,
6,0100201001 Amurrio sección 01001,,,,,,,,,,,,,,,
7,0100201002 Amurrio sección 01002,,,,,,,,,,,,,,,
8,0100201003 Amurrio sección 01003,,,,,,,,,,,,,,,
9,0100201004 Amurrio sección 01004,,,,,,,,,,,,,,,


In [21]:
#Extract only the columns that we are interested, on this case only Renta neta media por persona 2017,2016, 2015
atlas_socioeconimo = atlas_socioeconimo.iloc[:, [0, 1, 2, 3]]

#Change columnas name to get clear infomation

atlas_socioeconimo.columns = ['Index', '2017 Income', '2016 Income', '2015 Income']

#Change datatypes to filter with dropna ah use only the rows that contain information
atlas_socioeconimo['2017 Income'] = pd.to_numeric(atlas_socioeconimo['2017 Income'], errors='coerce').astype('Int64')
atlas_socioeconimo['2016 Income'] = pd.to_numeric(atlas_socioeconimo['2016 Income'], errors='coerce').astype('Int64')
atlas_socioeconimo['2015 Income'] = pd.to_numeric(atlas_socioeconimo['2015 Income'], errors='coerce').astype('Int64')

#Reduction of the df removing null values
atlas_socioeconimo = atlas_socioeconimo.dropna()


#Create the global variable taking in consideration all income in this period
atlas_socioeconimo['Global'] =atlas_socioeconimo['2017 Income']+atlas_socioeconimo['2016 Income']+atlas_socioeconimo['2015 Income']

# Take codes appart from location name because we need to do Joins in the future
#atlas_socioeconimo['NPRO'] = atlas_socioeconimo['Index'].str.split().str[-1]
atlas_socioeconimo['Index'] = atlas_socioeconimo['Index'].str.extract('(\d+)')


#Obtain incomes and codes
atlas_socioeconimo

,Index,2017 Income,2016 Income,2015 Income,Global,NPRO
0,01001,13281,13086,12936,39303,Alegría-Dulantzi
4,01002,13862,13691,13800,41353,Amurrio
13,01003,16177,17194,15786,49157,Aramaio
16,01004,12530,13185,12722,38437,Artziniega
19,01006,13115,11330,13040,37485,Armiñón
...,...,...,...,...,...,...
55082,5200108011,16433,16373,15847,48653,08011
55083,5200108012,17350,17185,16792,51327,08012
55084,5200108013,12553,12705,11823,37081,08013
55085,5200108014,8906,8872,8937,26715,08014


In [7]:
atlas_socioeconimo
atlas_socioeconimo['lens'] =  atlas_socioeconimo['Index'].apply(lambda x: len(str(x)))
print(atlas_socioeconimo['lens'].unique())

[ 5  7 10]


In [93]:
zonas_operador = gpd.read_file('/work/GPX programing test/zonas_operador.geojson')
print(zonas_operador.shape)
zonas_operador.dtypes


(113, 10)


Index            int64
Coloration       int64
C0             float64
C1             float64
ZE              object
Surface        float64
TauxConcav     float64
Lac-CellId      object
Population      object
geometry      geometry
dtype: object

In [94]:
def calcular_centroide(row):
    geometry_data = row['geometry']
    multipolygon = shape(geometry_data)
    centroid = multipolygon.centroid
    centroid_coords = (centroid.x, centroid.y)  # Convierte el centroide a una tupla de coordenadas
    return centroid_coords


# Aplica la función para calcular los centroides y crea la columna 'centroide'
zonas_operador['centroide'] = zonas_operador.apply(calcular_centroide, axis=1)

zonas_operador = zonas_operador[['Index', 'Population','centroide']]
zonas_operador.Index = zonas_operador.Index.astype('object')
#zonas_operador['lens'] =  zonas_operador['Index'].apply(lambda x: len(str(x)))



zonas_operador

,Index,Population,centroide
0,10113,17520,"(2.1569322495068772, 41.36446822555657)"
1,10115,38090,"(2.159359923207574, 41.37853652453478)"
2,10114,22000,"(2.1688545577838925, 41.373011178875345)"
3,10125,22923,"(2.172905132941295, 41.376540848891736)"
4,10124,14739,"(2.190197608860213, 41.377349237279056)"
...,...,...,...
108,1102,91083,"(2.03636781029606, 41.3430572426467)"
109,1159,66168,"(2.032205213860239, 41.302976277777645)"
110,949,46705,"(1.9837474451185597, 41.294668219954396)"
111,880,6961,"(1.9015356228294058, 41.31860011293308)"


In [95]:
zonas_censales_ine = gpd.read_file('/work/GPX programing test/zonas_censales_ine.json')
zonas_censales_ine

,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,CLAU2,NPRO,...,CNUT0,CNUT1,CNUT2,CNUT3,ESTADO,OBS,NMUN,Shape_Leng,Shape_Area,geometry
0,0100901001,01009,001,01,009,01,16,0100901,01009,Araba/Álava,...,ES,2,1,1,I,,Asparrena,51725.353538,6.326080e+07,"MULTIPOLYGON (((-2.25069 42.89550, -2.31610 42..."
1,0101001002,01010,002,01,010,01,16,0101001,01010,Araba/Álava,...,ES,2,1,1,I,,Ayala/Aiara,13350.774728,7.332951e+06,"POLYGON ((-2.97499 43.09904, -3.01602 43.11833..."
2,0103101001,01031,001,01,031,01,16,0103101,01031,Araba/Álava,...,ES,2,1,1,I,,Laguardia,87711.717051,8.041601e+07,"MULTIPOLYGON (((-2.52467 42.61471, -2.55285 42..."
3,0103301001,01033,001,01,033,01,16,0103301,01033,Araba/Álava,...,ES,2,1,1,I,,Lapuebla de Labarca,12331.494377,5.950453e+06,"POLYGON ((-2.54903 42.48445, -2.57448 42.50752..."
4,0103701001,01037,001,01,037,01,16,0103701,01037,Araba/Álava,...,ES,2,1,1,I,,Arraia-Maeztu,60761.315212,1.227207e+08,"POLYGON ((-2.36929 42.80425, -2.40900 42.80644..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36304,3501605017,35016,017,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",15642.540059,5.318911e+06,"POLYGON ((-15.49347 28.08960, -15.49699 28.096..."
36305,3501605009,35016,009,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",2929.633264,2.023513e+05,"POLYGON ((-15.48841 28.09245, -15.48927 28.093..."
36306,3501605033,35016,033,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",9796.053160,2.625522e+06,"POLYGON ((-15.48999 28.08495, -15.49145 28.086..."
36307,3501605034,35016,034,05,016,35,05,3501605,35016,"Palmas, Las",...,ES,7,0,5,M,,"Palmas de Gran Canaria, Las",1994.804489,1.635899e+05,"POLYGON ((-15.48999 28.08495, -15.48559 28.089..."


In [104]:


faltantes = []
ndf = []
for elemento in zonas_operador['Index']:
    resultado = zonas_censales_ine[zonas_censales_ine['CUSEC'].str.contains(str(elemento))]
    centroide_str = zonas_operador[zonas_operador['Index'] == elemento]['centroide']
    print(elemento, '\n \n \n \n')
    if resultado.shape[0] == 0:
        faltantes.append(elemento)
    else:
        centroide = centroide_str.iloc[0]  # Obtiene el valor numérico del centroide
        resultado['centroide'] = resultado.apply(calcular_centroide, axis=1)

        # Calcula la distancia euclidiana entre el centroide y cada valor en la columna 'centroide'
        resultado['distancia'] = resultado['centroide'].apply(lambda x: euclidean(centroide, x))
        
        # Encuentra el índice de la fila con la distancia mínima
        indice_fila_mas_cercana = resultado['distancia'].idxmin()
        
        # Imprime los datos de la fila más cercana
        ndf.append(resultado.loc[indice_fila_mas_cercana])
ndf



10113 
 
 
 

10115 
 
 
 

10114 
 
 
 

10125 
 
 
 

10124 
 
 
 

10127 
 
 
 

10126 
 
 
 

10121 
 
 
 

10120 
 
 
 

10123 
 
 
 

10122 
 
 
 

10133 
 
 
 

10132 
 
 
 

10134 
 
 
 

10135 
 
 
 

10129 
 
 
 

10131 
 
 
 

10130 
 
 
 

10141 
 
 
 

10140 
 
 
 

10143 
 
 
 

10142 
 
 
 

10137 
 
 
 

10136 
 
 
 

10139 
 
 
 

10138 
 
 
 

10149 
 
 
 

10151 
 
 
 

10150 
 
 
 

10145 
 
 
 

10144 
 
 
 

10147 
 
 
 

10146 
 
 
 

10157 
 
 
 

10156 
 
 
 

10159 
 
 
 

10158 
 
 
 

10153 
 
 
 

10152 
 
 
 

10155 
 
 
 

10165 
 
 
 

10154 
 
 
 

10164 
 
 
 

10166 
 
 
 

10167 
 
 
 

10161 
 
 
 

10148 
 
 
 

10160 
 
 
 

10163 
 
 
 

10162 
 
 
 

10169 
 
 
 

10168 
 
 
 

10003 
 
 
 

10171 
 
 
 

10183 
 
 
 

10184 
 
 
 

10182 
 
 
 

10181 
 
 
 

10128 
 
 
 

10190 
 
 
 

10189 
 
 
 

10192 
 
 
 

10191 
 
 
 

10186 
 
 
 

10185 
 
 
 

10188 
 
 
 

10187 
 
 
 

10198 
 
 
 

10197 
 
 
 

10200 
 
 
 

10199 
 
 
 

10194 

[CUSEC                                                0801910113
 CUMUN                                                     08019
 CSEC                                                        113
 CDIS                                                         10
 CMUN                                                        019
 CPRO                                                         08
 CCA                                                          09
 CUDIS                                                   0801910
 CLAU2                                                     08019
 NPRO                                                  Barcelona
 NCA                                                    Cataluña
 CNUT0                                                        ES
 CNUT1                                                         5
 CNUT2                                                         1
 CNUT3                                                         1
 ESTADO                  

In [111]:
columnas = ['Index', 'Population', 'CUSEC', 'NPRO', 'NCA']


faltantes= []


for elemento in zonas_operador['Index']:
    resultado = zonas_censales_ine[zonas_censales_ine['CUSEC'].str.contains(str(elemento))]
    centroide_str = zonas_operador[zonas_operador['Index'] == elemento]['centroide']
    population = zonas_operador[zonas_operador['Index'] == elemento]['Population']

    if resultado.shape[0] == 0:
        faltantes.append(elemento)
    else:
        centroide = centroide_str.iloc[0]  # Obtiene el valor numérico del centroide
        resultado['centroide'] = resultado.apply(calcular_centroide, axis=1)

        # Calcula la distancia euclidiana entre el centroide y cada valor en la columna 'centroide'
        resultado['distancia'] = resultado['centroide'].apply(lambda x: euclidean(centroide, x))
        
        # Encuentra el índice de la fila con la distancia mínima
        indice_fila_mas_cercana = resultado['distancia'].idxmin()
        
        # Obtiene los datos de la fila más cercana
        fila_mas_cercana = resultado.loc[indice_fila_mas_cercana]
        ndf.append([str(elemento),population,fila_mas_cercana['CUSEC'],fila_mas_cercana['NPRO'],fila_mas_cercana['NCA'] ])



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d79b2b15-9a76-4b8a-9139-6103e6385a17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>